#### **0. Chuẩn bị dữ liệu, cấu hình các tham số**

In [2]:
import re
import sys
sys.path.append('../')
import os
import pandas as pd
from collections import Counter
import datetime

pd.set_option('display.max_rows', None)

SETTING_PARAMS = {
    'Apache': {
        'log_file': './logs/Apache/Apache_full.log',
        'log_template': './logs/Apache/Apache_full.log_templates.csv',
        'log_structure': './logs/Apache/Apache_full.log_structured.csv',
        'log_format': '\[<Time>\] \[<Level>\] <Content>',
        'filters': [],
        'regexs': [r'\/(?:\w+\/){2,}\w+\.\w+$'],    
    },
    'HealthApp':{
        'log_file': './logs/HealthApp/HealthApp_full.log',
        'log_template': './logs/HealthApp/HealthApp_full.log_templates.csv',
        'log_structure': './logs/HealthApp/HealthApp_full.log_structured.csv',
        'log_format': '<Time>\|<Component>\|<Pid>\|<Content>',
        'filters': [],
        'regexs': [],  
    },
    'Mac': {
        'log_file': './logs/Mac/Mac_full.log',
        'log_template': './logs/Mac/Mac_full.log_templates.csv',
        'log_structure': './logs/Mac/Mac_full.log_structured.csv',
        'log_format': '<Month>  <Date> <Time> <User> <Component>\[<PID>\]( \(<Address>\))?: <Content>',
        'filters': [],
        'regexs': [r'([\w-]+\.){2,}[\w-]+'],  
    },
    'Linux': {
        'log_file': './logs/Linux/Linux_full.log',
        'log_template': './logs/Linux/Linux_full.log_templates.csv',
        'log_structure': './logs/Linux/Linux_full.log_structured.csv',
        'log_format': '<Month> <Date> <Time> <Level> <Component>(\[<PID>\])?: <Content>',
        'filters': [],
        'regexs': [r'(\d+\.){3}\d+', r'\d{2}:\d{2}:\d{2}'],  
    },
    'OpenSSH': {
        'log_file': './logs/OpenSSH/OpenSSH_full.log',
        'log_template': './logs/OpenSSH/OpenSSH_full.log_templates.csv',
        'log_structure': './logs/OpenSSH/OpenSSH_full.log_structured.csv',
        'log_format': '<Date> <Day> <Time> <Component> sshd\[<Pid>\]: <Content>',
        'filters': [],
        'regexs': [r"(\d+):"],    
    },
}

#### **1. Viết các phương thức sử dụng**

##### **1.1. Các phương thức sử dụng đọc dữ liệu**
- **`log_to_dataframe()`**
- **`generate_logformat_regex()`**

In [3]:
# ============ Các phương thức sử dụng để đọc dữ liệu ============ #
def log_to_dataframe(log_file, regex, headers):
    """ Phương thức chuyển đổi file log thành dataframe
    """
    log_messages = []
    linecount = 0
    with open(log_file, 'r', encoding="utf8") as fin:
        for line in fin.readlines():
            try:
                match = regex.search(line.strip())
                message = [match.group(header) for header in headers]
                log_messages.append(message)
                linecount += 1
            except Exception as e:
                pass
    logdf = pd.DataFrame(log_messages, columns=headers)
    logdf.insert(0, 'LineId', None)
    logdf['LineId'] = [i + 1 for i in range(linecount)]
    return logdf

def generate_logformat_regex(logformat):
    """ Phương thức tạo regex từ logformat, biểu thức định dạng của một event log: 
    Ex: 'log_format': '<Date> <Time> <Pid> <Level> <Component>: <Content>'
    """
    headers = []
    splitters = re.split(r'(<[^<>]+>)', logformat)
    regex = ''
    for k in range(len(splitters)):
        if k % 2 == 0:
            splitter = re.sub(' +', '\\\s+', splitters[k])
            regex += splitter
        else:
            header = splitters[k].strip('<').strip('>')
            regex += '(?P<%s>.*?)' % header
            headers.append(header)
    regex = re.compile('^' + regex + '$')
    return headers, regex

##### **1.2. Các phương thức sử dụng để tiền xử lý**
- **`pre_hasNumbers(s)`**
- **`pre_regexAndFilter(line, regexs = [], filters = [])`**

In [4]:
def pre_hasNumbers(s):
    return any(char.isdigit() for char in s)

def pre_regexAndFilter(line, regexs = [], filters = []):
    for currentFil in filters:
        line = re.sub(currentFil, '', line)
    for currentRex in regexs:
        line = re.sub(currentRex, "<*>", line)
    
    # tokens = line.strip().split()
    return line.strip()

#### **1.2. Thực hiện nhóm log** 

In [ ]:
from tqdm import tqdm
import string


class LogCluster:
    def __init__(self, keyGroup, logTemplate, tokens, length, logIDL=None, dynamic_tokenL=None, static_tokenL=None):
        self.keyGroup = keyGroup
        self.logTemplate = logTemplate
        self.tokens = tokens
        self.length = length
        self.logIDL = logIDL if logIDL is not None else []
        self.dynamic_tokenL = dynamic_tokenL if dynamic_tokenL is not None else {}
        self.static_tokenL = static_tokenL if static_tokenL is not None else {}
        
    def __str__(self):
        dynamic_str = "Dynamic Tokens: {\n"
        for k, v in self.dynamic_tokenL.items():
            dynamic_str += f"  {k}: {v},\n"
        dynamic_str += "}"
        
        static_str = "Cover Tokens: {\n"
        for k, v in self.static_tokenL.items():
            static_str += f"  {k}: {v},\n"
        static_str += "}"
        return (
            f"Key: {self.keyGroup}\n"
            f"Template: {self.logTemplate}\n"
            f"Tokens: {self.tokens}\n"
            f"Length: {self.length}\n"
            f"Len LogIDs: {len(self.logIDL)}\n"
            f"{dynamic_str}\n"
            f"{static_str}\n"
        )

    
class Node:
    def __init__(self, childD=None, depth=0, digitOrtoken=None):
        if childD is None:
            childD = dict()
        self.childD = childD
        self.depth = depth
        self.digitOrtoken = digitOrtoken
    

def compare_templates(datasets, parse_df):
    structured_df = pd.read_csv(datasets['log_structure']) 
    unique_templates = structured_df['EventTemplate'].unique()
    print(f"SHAPE: {structured_df.shape}")
    print(f"SHAPE PARSER: {parse_df.shape}")
    print(f"Num of templates: {len(unique_templates)}")

    template_compare = {}
    for template in tqdm(unique_templates, total=len(parse_df), desc="Processing templates", unit="rows"):
        arr_index = structured_df[structured_df['EventTemplate'] == template].index.tolist()
        parse_template_series = parse_df.loc[arr_index, 'ParseTemplate1']
        parse_template_unique = parse_template_series.unique().tolist()

        hash_key = hash(template)
        template_compare[hash_key] = {
            'ground_truth': template,
            'parse': parse_template_unique,
            'index': arr_index,
            'length': len(template.strip().split()),
            'nums': len(arr_index)
        }
        
    sorted_items = sorted(
            template_compare.items(),
            key=lambda item: (item[1]['length'], item[1]['ground_truth'])
        )

    num_dif = 0
    for idx, (key, value) in enumerate(sorted_items, 1):
        if len(value['parse']) != 1 or value['parse'][0] != value['ground_truth']:
            num_dif += 1
            print(f"No. {idx}")
            print(f"Length: {value['length']}, Nums: {value['nums']}")
            print(f"Ground truth  : {value['ground_truth']}")
            print(f"Parse templs  : {value['parse']}")
            print(f"Length parse: {len(value['parse'])}")
            print("-" * 40)
    print(f"Total differences found: {num_dif}")


def get_common_separators(strings, separators):
    """Trả về danh sách các ký tự đặc biệt có trong tất cả chuỗi."""
    common = set(separators)
    for s in strings:
        common &= set([c for c in s if c in separators])
    return list(common)

def tokenize_keep_separator(s, seps):
    """Tách chuỗi giữ lại ký tự phân tách."""
    if not seps:
        return [s]
    pattern = '|'.join(map(re.escape, seps))
    tokens = re.split(f'({pattern})', s)
    return [tok for tok in tokens if tok]





def log_cluster_0(datasets, dic_special_token=['true', 'false']):
    """ Phương thức chuyển đổi file log thành dataframe
    """
    log_headers, log_regex = generate_logformat_regex(datasets['log_format'])
    logs_df = log_to_dataframe(datasets['log_file'], log_regex, log_headers)
    
    # ============ Bước tiền xử lý bởi regex và filter ============ #
    parse_df = logs_df.copy()
    parse_df['ParseTemplate0'] = ["" for _ in range(len(parse_df))]                                # Sau bước tiền xử lý regex, filters
    parse_df['Tokens0'] = [[] for _ in range(len(parse_df))]               # Arr tokens sau khi xử lý regex, filters
    parse_df['ParseTemplate1'] = ["" for _ in range(len(parse_df))]        # Template sau bước xử lý has_Numbers
    parse_df['Tokens1'] = [[] for _ in range(len(parse_df))]               # Arr tokens sau khi xử lý has_Numbers và dic_special_token
    
    # Tạo biểu thức chính quy: nếu có ký tự đặc biệt ngay trước từ khóa
    # VD: '=true', ':False' → '<*>'
    keywords = '|'.join([re.escape(k) for k in dic_special_token])
    patterns_2 = re.compile(rf"([{re.escape(string.punctuation)}])({keywords})\b", flags=re.IGNORECASE)
    
    for idx, line in tqdm(parse_df.iterrows(), desc="Tiền xử lý giai đoạn 0 và 1"):
        parsed = pre_regexAndFilter(line['Content'], regexs=datasets['regexs'], filters=datasets['filters'])
        tokens0 = str(parsed).strip().split()
    
        parse_df.at[idx, 'ParseTemplate0'] = parsed
        parse_df.at[idx, 'Tokens0'] = tokens0
        
        groups_token = []
        tokens1 = []
        for token in tokens0:
            if token == "<*>":
                tokens1.append("<*>")
            else: 
                if (not pre_hasNumbers(token) and str(token).lower() not in dic_special_token):
                    temp = re.sub(patterns_2, r'\1<*>', token)
                    groups_token.append(temp)
                    tokens1.append(temp)
                else:
                    tokens1.append("<*>")
        
        parse_df.at[idx, 'Tokens1'] = tokens1
        parse_df.at[idx, 'ParseTemplate1'] = f"{' '.join(groups_token)} , {len(tokens0)}"
        
    
        
    # parse_df['ParseTemplate'] = parse_df['Content'].apply(lambda x: pre_regexAndFilter(x, regexs=datasets['regexs'], filters=datasets['filters']))
    # ============================ END ============================ #
    
    # ===================== Bước phân cụm log ===================== #
    unique_logs_0 = parse_df['ParseTemplate0'].unique()
    print(len(unique_logs_0))
    
    # Sử dụng templates 1 để giải quyết vấn đề
    unique_logs_1 = parse_df.groupby("ParseTemplate1")
    print(len(unique_logs_1.groups.keys()))
    
    log_clusters_list = []
    for key, group in unique_logs_1:
        # Lấy dòng đầu tiên để làm đại diện template cho nhóm
        sample_rows = group.head(100)
        tokens = sample_rows.iloc[0]['Tokens1']
        logTemplate = " ".join(tokens)
        length = len(tokens)
        logIDL = group.index.tolist()  # danh sách index thuộc nhóm này
        
        dynamic_tokenL = {}
        for pos, token in enumerate(tokens):
            if "<*>" in token:
                values = []
                for _, row in sample_rows.iterrows():
                    values.append(row['Content'].strip().split()[pos])
                dynamic_tokenL[pos] = values
        
        cluster = LogCluster(
            keyGroup=key,
            logTemplate=logTemplate,
            tokens=tokens,
            length=length,
            logIDL=logIDL,
            dynamic_tokenL= dynamic_tokenL.copy()
        )
        log_clusters_list.append(cluster)

    # ========================= Xử lý phân tách dynamic_tokens ==============================
    # Phân tách bởi các giá trị đặc biệt, sau đó đi so khớp.
    # Khai báo các dấu phân tách
    punctionL = '(),<>:;{}[]~='
    # to_remove = ".#%$"
    # custom_punctuation = ''.join(c for c in string.punctuation if c not in to_remove)
    
    for cluster in log_clusters_list:
        # Phân tách:
        result = {}
        for key, values in cluster.dynamic_tokenL.items():
            # Tìm ký tự phân tách chung
            common_seps = get_common_separators(values, punctionL)

            # Tách và lưu các token giữ nguyên thứ tự
            tokenized_list = [tokenize_keep_separator(s, common_seps) for s in values]

            # Xoay list theo cột để xử lý theo vị trí
            transposed = list(zip(*tokenized_list))

            processed_tokens = []
            for idx, tokens_at_pos in enumerate(transposed):
                # Nếu có số hoặc token đặc biệt, chuyển thành <*>
                transformed = []
                for tok in tokens_at_pos:
                    if tok.lower() in dic_special_token or re.search(r'\d', tok):
                        transformed.append('<*>')
                    else:
                        transformed.append(tok)
                # Nếu >3 giá trị unique khác nhau thì cũng chuyển hết thành <*>
                if len(set(transformed)) > 3:
                    processed_tokens.append(['<*>'] * len(tokens_at_pos))
                else:
                    processed_tokens.append(transformed)

            # Xoay lại hàng
            recombined = [''.join(toks) for toks in zip(*processed_tokens)]
            result[key] = list(set(recombined))
        cluster.static_tokenL = result.copy()
    
    
    
    
    for cluster in log_clusters_list:
        print(cluster)
        print("="*80)

    
    
    
    
    
    # for template, group_df in unique_logs_1:
    #     print(f"Template: {template}")
    #     print(len(group_df['LineId']))  # hoặc group_df[['Content']] nếu muốn giữ dạng bảng
    #     print('-' * 50)
    
    # Xử lý các tokens động:
    
    
    compare_templates(datasets, parse_df)
    
    
    return parse_df, log_clusters_list

parse_df, log_clusters_list = log_cluster_0(SETTING_PARAMS['Mac'])



In [6]:
from collections import defaultdict

# Thử xử lý với các cụm cluster khác nhau:
cluster_0 = log_clusters_list[0]
print(cluster_0)

punctionL = '(),<>:;{}[]~='

result = {}
for key, values in cluster_0.dynamic_tokenL.items():
    dic_special_token = ['true', 'false']
    
    # Tìm ký tự phân tách chung
    common_seps = set(punctionL)
    
    # Tách và lưu các token giữ nguyên thứ tự
    tokenized_list = [tokenize_keep_separator(s, common_seps) for s in values]
    
    # ================= Thực hiện nhóm theo độ dài =====================
    grouped_results = defaultdict(list)
    for key1, values in cluster_0.dynamic_tokenL.items():
        # Nhóm theo độ dài
        length_groups = defaultdict(list)
        for tokens in tokenized_list:
            length_groups[len(tokens)].append(tokens)
        
        for length, group in length_groups.items():
            first_token_map = defaultdict(list)
            tokens_with_number = []

            for tokens in group:
                first_token = tokens[0]
                if pre_hasNumbers(first_token):
                    tokens_with_number.append(tokens)
                else:
                    first_token_map[first_token].append(tokens)
            
            num_types = len(first_token_map)

            if num_types <= 3:
                # Nếu chỉ có 1 hoặc 2 loại token không chứa số → tách riêng
                for first_token, sub_group in first_token_map.items():
                    key_name = f"{length} {first_token}"
                    grouped_results[key_name].extend(sub_group)
            else:
                # Nếu có đúng 4 loại → gộp tất cả vào nhóm <*>
                for sub_group in first_token_map.values():
                    grouped_results[f"{length} <*>"].extend(sub_group)

            # Gộp token có chứa số vào nhóm <*>
            if tokens_with_number:
                grouped_results[f"{length} <*>"].extend(tokens_with_number)
    # ========================== END ==================================
    
    result[key] = []
    for key_group, values_group in grouped_results.items():
        # Xoay list theo cột để xử lý theo vị trí
        tokenized_list = values_group
        transposed = list(zip(*tokenized_list))

        processed_tokens = []
        for idx, tokens_at_pos in enumerate(transposed):
            # Nếu có số hoặc token đặc biệt, chuyển thành <*>
            transformed = []
            for tok in tokens_at_pos:
                if tok.lower() in dic_special_token or re.search(r'\d', tok):
                    transformed.append('<*>')
                else:
                    transformed.append(tok)
            # Nếu >3 giá trị unique khác nhau thì cũng chuyển hết thành <*>
            if len(set(transformed)) > 3:
                processed_tokens.append(['<*>'] * len(tokens_at_pos))
            else:
                processed_tokens.append(transformed)

        # Xoay lại các phần tử (transpose hàng thành cột)
        recombined = [''.join(col) for col in zip(*processed_tokens)]

        # Lọc giá trị duy nhất
        unique_recombined = list(set(recombined))
        print(len(unique_recombined), unique_recombined)

        # Xử lý theo số lượng recombined
        if len(unique_recombined) <= 3:
            for temp in unique_recombined:
                result[key].append(temp)
        else:
            result[key].append("<*>") if "<*>" not in result[key] else None

        
    # for text in tokenized_list:
    #     print(text, len(text))
    
    # for key, values in grouped_results.items():
    #     print(f"{key:30}: {values}")
    print(result)



Key:  , 1
Template: <*>
Tokens: ['<*>']
Length: 1
Len LogIDs: 196
Dynamic Tokens: {
  0: ['kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'newHeight=5421.000000,oldHeight=5421.000000', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'kern_open_file_for_direct_io(0)', 'IOHibernatePollerOpen(0)', 'kern_open_file_for_direct_io(0)', 'newHeight=990.000000,oldHeight=5452.